In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import AdamW

2024-02-19 19:42:36.083343: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
#改正

from transformers import TextClassificationPipeline
import torch.nn as nn
from transformers import pipeline
from transformers import TextClassificationPipeline
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 使用するモデルを指定して、トークナイザとモデルを読み込む
checkpoint = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model_kanjyou = torch.load('../../kanjyou/multi_pos/multi_pos_weight_.pth')


class TextClassificationPipelineAddOutputs(TextClassificationPipeline):
    
    def postprocess(self, model_outputs, function_to_apply=None, return_all_scores=False):
        # 既存の処理を行います
        processed = super().postprocess(model_outputs, function_to_apply, return_all_scores)
        
        logits = model_outputs['logits']
        
        logits =torch.sigmoid(logits)
        
        if return_all_scores:
            return [{**proc, **{'logits':logits}} for proc in processed]
        else:
            #print(processed)
            def get_score(data: dict):
                return data['score']
            if len(processed) > 1:
                max_proc = max(processed, key=get_score)
            else:
                max_proc = processed[0]
            return {**max_proc, **{'logits':logits}}
            #return [{**proc, **{'logits':logits}} for proc in processed]
            

#sentiment_analyzer = pipeline("sentiment-analysis",model = model.to("cpu"), tokenizer = tokenizer)
sentiment_analyzer =TextClassificationPipelineAddOutputs(model = model_kanjyou.to("cpu"), tokenizer = tokenizer)

num_labels= {
        "LABEL_0":'喜び',
        "LABEL_1":'悲しみ',
        "LABEL_2":'期待',
        "LABEL_3":'驚き',
        "LABEL_4":'怒り',
        "LABEL_5":'恐れ',
        "LABEL_6":'嫌悪',
        "LABEL_7":'信頼',
}

result = sentiment_analyzer("楽しいね。")

print(num_labels[result[0]['label']])


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


喜び


In [4]:
# from transformers import TextClassificationPipeline
# import torch.nn as nn

# from transformers import pipeline
# from transformers import TextClassificationPipeline

# model_name = "cl-tohoku/bert-large-japanese"
# # model_name = "cl-tohoku/bert-base-japanese-v2"

# # model = BertForSequenceClassification.from_pretrained(model_name, num_labels=8,problem_type="multi_label_classification")
# # model = model.to(device)
# tokenizer = BertTokenizer.from_pretrained(model_name)

# class TextClassificationPipelineAddOutputs(TextClassificationPipeline):
    
#     def postprocess(self, model_outputs, function_to_apply=None, return_all_scores=False):
#         # 既存の処理を行います
#         processed = super().postprocess(model_outputs, function_to_apply, return_all_scores)
        
#         logits = model_outputs['logits']
        
#         logits =torch.sigmoid(logits)
        
#         if return_all_scores:
#             return [{**proc, **{'logits':logits}} for proc in processed]
#         else:
#             #print(processed)
#             def get_score(data: dict):
#                 return data['score']
#             if len(processed) > 1:
#                 max_proc = max(processed, key=get_score)
#             else:
#                 max_proc = processed[0]
#             return {**max_proc, **{'logits':logits}}



# modell = torch.load('../../kanjyou/multi_pos/multi_pos_weight_.pth')

# #sentiment_analyzer = pipeline("sentiment-analysis",model = model.to("cpu"), tokenizer = tokenizer)
# sentiment_analyzer =TextClassificationPipelineAddOutputs(model = modell.to("cpu"), tokenizer = tokenizer)

# num_labels= {
#         "LABEL_0":'喜び',
#         "LABEL_1":'悲しみ',
#         "LABEL_2":'期待',
#         "LABEL_3":'驚き',
#         "LABEL_4":'怒り',
#         "LABEL_5":'恐れ',
#         "LABEL_6":'嫌悪',
#         "LABEL_7":'信頼',
# }



In [5]:
result = sentiment_analyzer("今日は疲れた")
#print(msg)

kanjyou = num_labels[result[0]['label']]
print(kanjyou)


悲しみ


In [6]:
from pathlib import Path
# モデルのディレクトリ
# strで指定すると、from_pretrainedでなぜかHuggingFaceを参照したため、PathLikeで指定した。
output = Path("/home/n.yumeka/workspace/wrime/gakusyu/output_ariari2")  
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained(output)
model.to(device)
model.eval()  

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [27]:
def generate_reply(inp,kanjyou,taiwakoui, num_gen=1):
    input_text = "<s>" + "(" + str(kanjyou) + ")" + "[SEP]"   + str(inp) + "[SEP]" + "("+ str(taiwakoui) + ")"+ "[SEP]" 
    #input_text = "<s>" + "(" + str(kanjyou) + ")" + str(inp) + "[SEP]"
    input_ids = tokenizer.encode( input_text, return_tensors='pt').to(device)
    # 生成時のパラメータ変更
    out = model.generate(input_ids, do_sample=True, max_length=30, num_return_sequences=num_gen, top_p=0.9, top_k=10, bad_words_ids=[[1], [5]], no_repeat_ngram_size=1)
    # out = model.generate(input_ids, do_sample=True, max_length=30, num_return_sequences=10, top_p=0.8, top_k=40, temperature=0.6,  bad_words_ids=[[1], [5]], no_repeat_ngram_size=1)

    #print(">", "あなた")
    #print(inp)
    #print(">", "システム")
    for sent in tokenizer.batch_decode(out):
        sent = sent.split('[SEP]</s>')[1]
        sent = sent.replace('<unk>', '')
        sent = sent.replace('</s>', '')
        sent = sent.replace('(.*?)','')
        sent = sent.strip('w')
        #print(sent)
        return sent

In [28]:
import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

class DialogueAnalyzer:
    def __init__(self, model_path, model_2, dialogue_labels):
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        
        self.model = torch.load(model_2)  # モデルが torch.save で保存されていると仮定
        
        self.dialogue_labels = dialogue_labels
        

    @staticmethod
    def np_softmax(x):
        f_x = np.exp(x) / np.sum(np.exp(x))
        return f_x

    def analyze_dialogue(self, text1, text2):
        # モデルを評価モードに設定
        
        self.model.eval()
        
        # 入力データ変換 + 推論
        tokens = self.tokenizer(text1, text2, truncation=True, return_tensors="pt")
        
        tokens.to(self.model.device)
        preds = self.model(**tokens)

        # Softmax 確率の計算
        prob = self.np_softmax(preds.logits.cpu().detach().numpy()[0])

        # ラベルとその確率を持つ辞書を作成
        out_dict = {n: p for n, p in zip(self.dialogue_labels, prob)}

        # 最大確率を持つラベルを取得
        max_key = max(out_dict, key=out_dict.get)

        return max_key

# クラスの使用例
model_path = 'cl-tohoku/bert-base-japanese-whole-word-masking'

dialogue_labels = ["自己開示", "質問(YesNo)", '質問(What)', '応答(YesNo)', '応答(平叙)', 'あいづち', 'フィラー', '確認', '要求',"共感"]
model_2 ='../../dialogueact/data1//class_weight/dialogueact_class_weight_kyoukan.pth'

analyzer = DialogueAnalyzer(model_path, model_2, dialogue_labels)

# result = analyzer.analyze_dialogue("今日は疲れたな", "ほんとほんと")
# print(result)


In [ ]:
# 一行ずつファイルを読み込み、一部分を取り出す
import re
with open("../data/arinasi/arinasi1_test.txt", 'r') as file:
    text = file.read()
    # かっこを全て削除
    text = re.sub(r',', '、', text)
    text = re.sub(r'，', '、', text)
    text = re.sub(r'<s>', ',', text)
    text = re.sub(r'\[SEP\]', ',', text)
    text = re.sub(r'</s>', ',', text)
    #print(text)

with open('ariari_kakou1.txt', 'w', encoding='utf-8') as f:
    f.write(text)  
     
        

In [4]:
import pandas as pd
kaiwa = pd.read_table('ariari_kakou1.txt', sep = ',',encoding='utf-8')
kaiwa.columns = ['a', '発話', '応答','b']
# kaiwa
hatuwa = kaiwa["発話"]
hatuwa_list = hatuwa.tolist()

In [18]:
msg = input("あなた: ")


# 入力された値を表示


# print("入力された値:", user_input)関西出身なんですってね。


あなた:  昨日のデート、彼氏が寝坊して遅刻してデートに来なかったんですよ。


In [30]:
import random


# ユーザーに対してメッセージを表示し、入力を受け取る



result = sentiment_analyzer(msg)



print(msg)

kanjyou = num_labels[result[0]['label']]
print(kanjyou)
if kanjyou == "驚き":
    #taiwakoui = "質問(YesNo)"
        
    messages = ["質問(YesNo)", "共感"]
    probabilities = [7, 3]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
    
elif kanjyou == "怒り":
    #taiwakoui = "自己開示"
    
    messages = ["自己開示", "共感", "質問(What)","あいづち"]
    probabilities = [4, 3,2,1]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]    
    
elif kanjyou == "悲しみ":
    #taiwakoui = "自己開示"
    
    messages = ["自己開示","あいづち" ,"共感"]
    probabilities = [5, 2,3]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
    
elif kanjyou == "喜び":
    # taiwakoui = "質問(What)"
    messages = ["質問(What)", "共感"]
    probabilities = [5, 5]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
    
elif kanjyou == "期待":
    # taiwakoui = "質問(YesNo)"
    messages = ["質問(YesNo)", "共感"]
    probabilities = [6, 4]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
    
elif kanjyou == "信頼":
    # taiwakoui = "質問(YesNo)"
    messages = ["質問(YesNo)", "共感"]
    probabilities = [5, 5]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
elif kanjyou == "嫌悪":
    # taiwakoui = "自己開示"
    messages = ["自己開示", "共感","あいづち"]
    probabilities = [5, 3,2]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
elif kanjyou == "恐れ":
    #taiwakoui = "質問(What)"
    messages = ["質問(What)", "共感","質問(YesNo)","あいづち"]
    probabilities = [2,4,2,1]

    # ランダムに1つ選択
    taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
print(taiwakoui)
syoki = 0
while True:
    outou = generate_reply(msg,kanjyou,taiwakoui)
    # print(outou)
    dialogue = analyzer.analyze_dialogue(msg,outou)
    syoki = syoki + 1
    if dialogue == taiwakoui:
        print(outou)
        break
            
    if syoki >= 10 and syoki < 30:

        if dialogue == "共感" or dialogue == "あいづち":
            # print(syoki)
            print(outou)
            # anss.append(outou)
            
            break

    # if syoki >= 30 and syoki < 100:
    #     if dialogue == "自己開示" :
    #         # print(syoki)
    #         print(outou)
    #         break

    if  syoki >= 30:

        # anss.append("応答なし")
        print(outou)
        
        
        break
        
        
  

     
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input length of input_ids is 32, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input len

昨日のデート、彼氏が寝坊して遅刻してデートに来なかったんですよ。
悲しみ
自己開示


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input length of input_ids is 32, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input len

でも


In [146]:
# syoki = 0
# while True:
#     outou = generate_reply(msg,kanjyou,taiwakoui)
#     # print(outou)
#     dialogue = analyzer.analyze_dialogue(msg,outou)
#     syoki = syoki + 1
#     if dialogue == taiwakoui:
#         print(outou)


#         brea
#     if syoki > 10:
#         print(dialogue)
#         if dialogue == "共感" or dialogue == "あいづち":
            
#             print("×" + outou)
#             break
            

In [149]:
# import random
# def randam():
    
#     messages = ["ごめん", "ありがとう"]
#     probabilities = [7, 3]

#     # ランダムに1つ選択
#     return(random.choices(messages, weights=probabilities, k=1)[0])
# for i in range(30):
#     k=randam()
#     print(k)

In [10]:
len(hatuwa_list)

485

In [11]:
import pandas as pd
import random
#df.to_csv('kyoukanrei.csv', index = False)
msgg = []
anss = []


maetaiwakoui = []
kaisuu = []

for f in range(len(hatuwa_list)):
    
    
    result = sentiment_analyzer(hatuwa_list[f])
    # print(hatuwa_list[f])

    kanjyou = num_labels[result[0]['label']]
    print(f)
    # print(kanjyou)
    if kanjyou == "驚き":
        #taiwakoui = "質問(YesNo)"

        messages = ["質問(YesNo)", "共感"]
        probabilities = [7, 3]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]

    elif kanjyou == "怒り":
        #taiwakoui = "自己開示"

        messages = ["自己開示", "共感", "質問(What)","あいづち"]
        probabilities = [4, 3,2,1]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]    

    elif kanjyou == "悲しみ":
        #taiwakoui = "自己開示"

        messages = ["自己開示","あいづち" ,"共感"]
        probabilities = [5, 2,3]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]

    elif kanjyou == "喜び":
        # taiwakoui = "質問(What)"
        messages = ["質問(What)", "共感"]
        probabilities = [5, 5]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]

    elif kanjyou == "期待":
        # taiwakoui = "質問(YesNo)"
        messages = ["質問(YesNo)", "共感"]
        probabilities = [6, 4]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]

    elif kanjyou == "信頼":
        # taiwakoui = "質問(YesNo)"
        messages = ["質問(YesNo)", "共感"]
        probabilities = [5, 5]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
    elif kanjyou == "嫌悪":
        # taiwakoui = "自己開示"
        messages = ["自己開示", "共感","あいづち"]
        probabilities = [5, 3,2]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
    elif kanjyou == "恐れ":
        #taiwakoui = "質問(What)"
        messages = ["質問(What)", "共感","質問(YesNo)","あいづち"]
        probabilities = [2,4,2,1]

        # ランダムに1つ選択
        taiwakoui = random.choices(messages, weights=probabilities, k=1)[0]
    maetaiwakoui.append(taiwakoui)

    syoki = 0
    while True:
        
        outou = generate_reply(hatuwa_list[f],kanjyou,taiwakoui)
        # print(outou)
        dialogue = analyzer.analyze_dialogue(hatuwa_list[f],outou)
        
        syoki = syoki + 1

        if dialogue == taiwakoui:
            # print(outou)
            anss.append(outou)
            break
        if syoki >= 10 and syoki < 30:
            
            if dialogue == "共感" or dialogue == "あいづち":
                # print(syoki)
                #print("×" + outou)
                anss.append(outou)
                break
                
#         if syoki >= 30 and syoki < 100:
#             if dialogue == "自己開示" :
#                 print(syoki)
#                 anss.append(outou)
#                 break
                
#         if  syoki > 100:
           
#             anss.append("応答なし")
#             print("応答なし")
#             break
            
        if  syoki >= 30:

            # anss.append("応答なし")
            # print(outou)
            anss.append(outou)


            break

            
    # print(syoki)
    kaisuu.append(syoki)
    
                


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

1
2
3


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

...

480


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

481
482


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input length of input_ids is 32, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input len

483
484


In [12]:
maetaiwakoui
kaisuu

[9,
 1,
 10,
 1,
 1,
 4,
 10,
 11,
 10,
 30,
 11,
 30,
 13,
 11,
 30,
 1,
 3,
 10,
 1,
 1,
 10,
 10,
 1,
 1,
 1,
 11,
 1,
 30,
 10,
 10,
 10,
 10,
 10,
 1,
 1,
 30,
 12,
 30,
 30,
 2,
 5,
 5,
 3,
 10,
 30,
 10,
 10,
 1,
 2,
 10,
 3,
 10,
 6,
 9,
 30,
 7,
 1,
 10,
 10,
 10,
 17,
 30,
 1,
 1,
 3,
 10,
 10,
 3,
 1,
 10,
 10,
 1,
 30,
 10,
 30,
 10,
 10,
 2,
 1,
 30,
 30,
 30,
 13,
 10,
 13,
 30,
 10,
 5,
 11,
 2,
 9,
 15,
 10,
 1,
 11,
 4,
 10,
 10,
 2,
 10,
 30,
 30,
 20,
 30,
 30,
 30,
 10,
 10,
 11,
 10,
 5,
 10,
 5,
 5,
 2,
 7,
 5,
 1,
 2,
 1,
 5,
 10,
 13,
 30,
 12,
 13,
 30,
 4,
 1,
 7,
 10,
 5,
 1,
 2,
 30,
 30,
 1,
 5,
 5,
 10,
 10,
 10,
 10,
 10,
 10,
 30,
 1,
 2,
 10,
 2,
 1,
 10,
 18,
 10,
 10,
 10,
 2,
 1,
 5,
 1,
 2,
 10,
 6,
 6,
 11,
 10,
 10,
 10,
 12,
 8,
 30,
 10,
 6,
 10,
 3,
 4,
 30,
 10,
 1,
 1,
 1,
 1,
 10,
 10,
 10,
 2,
 10,
 10,
 30,
 1,
 10,
 3,
 30,
 10,
 27,
 30,
 30,
 10,
 3,
 5,
 10,
 30,
 19,
 30,
 3,
 1,
 30,
 10,
 30,
 11,
 1,
 14,
 1,
 10,
 5,
 10,
 10,
 13

In [15]:
kekka_bunseki = pd.DataFrame((zip(maetaiwakoui, kaisuu)), columns = ["対話行為", "応答数"])
kekka_bunseki
kekka_bunseki.to_excel('kekka_bunseki.xlsx', index = False)

In [16]:
len(anss)

485

In [18]:
# flat_list1 = [item for sublist in msgg for item in sublist]  
# flat_list2 = [item for sublist in anss for item in sublist]

d = pd.DataFrame(hatuwa_list, columns = ["ユーザ"])
df = pd.DataFrame(anss, columns=["システム"])
df = pd.concat([d, df], axis = 1)

df


#with open('kyoukanrei.csv', mode = 'a', encoding='utf-8') as f:
    #f.write(df)
# df.to_csv("filtering_arinashi.csv", index = False, encoding = "UTF-8")
df.to_excel('ariari_outou.xlsx', index = False)
#df.to_csv("kyoukanrei.csv", mode='a', index = False, encoding = "UTF-8")